# Learning a Reward Function using Preference Comparisons

The preference comparisons algorithm learns a reward function by comparing trajectory segments to each other.

To set up the preference comparisons algorithm, we first need to set up a lot of its internals beforehand:

In [1]:
from imitation.algorithms import preference_comparisons
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.policies.base import FeedForward32Policy, NormalizeFeaturesExtractor
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO

venv = DummyVecEnv([lambda: gym.make("Pendulum-v1")] * 8)

reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)

fragmenter = preference_comparisons.RandomFragmenter(warning_threshold=0, seed=0)
gatherer = preference_comparisons.SyntheticGatherer(seed=0)
reward_trainer = preference_comparisons.CrossEntropyRewardTrainer(
    model=reward_net,
    epochs=3,
)

agent = PPO(
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=venv,
    seed=0,
    n_steps=2048 // venv.num_envs,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
)

trajectory_generator = preference_comparisons.AgentTrainer(
    algorithm=agent,
    reward_fn=reward_net,
    exploration_frac=0.0,
    seed=0,
)

pref_comparisons = preference_comparisons.PreferenceComparisons(
    trajectory_generator,
    reward_net,
    fragmenter=fragmenter,
    preference_gatherer=gatherer,
    reward_trainer=reward_trainer,
    comparisons_per_iteration=100,
    #comparison_queue_size=1000,
    fragment_length=100,
    transition_oversampling=1,
    initial_comparison_frac=0.1,
    allow_variable_horizon=False,
    seed=0,
    initial_epoch_multiplier=1,
)

Then we can start training the reward model. Note that we need to specify the total timesteps that the agent should be trained and how many fragment comparisons should be made.

In [2]:
pref_comparisons.train(
    total_timesteps=10_000,
    total_comparisons=500,
)

Requested 20000 transitions but only 0 in buffer. Sampling 20000 additional transitions.
Creating fragment pairs
Gathering preferences
Dataset now contains 100 comparisons


Training reward model: 100%|██████████| 3/3 [00:44<00:00, 14.88s/it]

Training agent for 2500 timesteps


----------------------------------
| raw/                    |      |
|    agent/time/fps       | 407  |
|    agent/time/iterat... | 1    |
|    agent/time/time_e... | 5    |
|    agent/time/total_... | 2048 |
----------------------------------
------------------------------------------
| raw/                    |              |
|    agent/rollout/ep_... | -3.9         |
|    agent/time/fps       | 365          |
|    agent/time/iterat... | 2            |
|    agent/time/time_e... | 11           |
|    agent/time/total_... | 4096         |
|    agent/train/appro... | 0.0012673859 |
|    agent/train/clip_... | 0.2          |
|    agent/train/entro... | -1.43        |
|    agent/train/expla... | -0.0589      |
|    agent/train/learn... | 0.0003       |
|    agent/train/loss     | 0.258        |
|    agent/train/n_upd... | 10           |
|    agent/train/polic... | -0.000973    |
|    agent/train/std      | 1.01         |
|    agent/train/value... | 0.752        |
------------------------

Training reward model: 100%|██████████| 3/3 [01:29<00:00, 29.83s/it]

...

------------------------------------------
| raw/                    |              |
|    agent/rollout/ep_... | 0.463        |
|    agent/time/fps       | 424          |
|    agent/time/iterat... | 1            |
|    agent/time/time_e... | 4            |
|    agent/time/total_... | 10240        |
|    agent/train/appro... | 0.0054630716 |
|    agent/train/clip_... | 0.2          |
|    agent/train/entro... | -1.39        |
|    agent/train/expla... | 0.823        |
|    agent/train/learn... | 0.0003       |
|    agent/train/loss     | 0.408        |
|    agent/train/n_upd... | 40           |
|    agent/train/polic... | -0.00335     |
|    agent/train/std      | 0.957        |
|    agent/train/value... | 1.05         |
------------------------------------------
------------------------------------------
| raw/                    |              |
|    agent/rollout/ep_... | 0.569        |
|    agent/time/fps       | 373          |
|    agent/time/iterat... | 2            |
|    agent/

Training reward model: 100%|██████████| 3/3 [02:58<00:00, 59.56s/it]

Training agent for 2500 timesteps


------------------------------------------
| raw/                    |              |
|    agent/rollout/ep_... | -0.898       |
|    agent/time/fps       | 419          |
|    agent/time/iterat... | 1            |
|    agent/time/time_e... | 4            |
|    agent/time/total_... | 14336        |
|    agent/train/appro... | 0.0040166993 |
|    agent/train/clip_... | 0.2          |
|    agent/train/entro... | -1.36        |
|    agent/train/expla... | 0.976        |
|    agent/train/learn... | 0.0003       |
|    agent/train/loss     | 0.0956       |
|    agent/train/n_upd... | 60           |
|    agent/train/polic... | -0.00204     |
|    agent/train/std      | 0.933        |
|    agent/train/value... | 0.224        |
------------------------------------------
-----------------------------------------
| raw/                    |             |
|    agent/rollout/ep_... | -1.6        |
|    agent/time/fps       | 372         |
|    agent/time/iterat... | 2           |
|    agent/time/

{'reward_loss': 0.0, 'reward_accuracy': 0.0}

In [8]:
eval_env = gym.make("Pendulum-v1")
obs = eval_env.reset()
rews = []
dones = []

for _ in range(10000):
    action, _ = agent.predict(obs)
    obs, rew, done, _ = eval_env.step(action)
    if done:
        obs = eval_env.reset()
    
    rews.append(rew)
    dones.append(done)
    # frame = eval_env.render('rgb_array')
    # frames.append(frame)


/home/norabelrose/miniforge3/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [9]:
import numpy as np
np.sum(dones)

20

After we trained the reward network using the preference comparisons algorithm, we can wrap our environment with that learned reward.

In [3]:
from imitation.rewards.reward_wrapper import RewardVecEnvWrapper


learned_reward_venv = RewardVecEnvWrapper(venv, reward_net.predict)

Now we can train an agent, that only sees those learned reward.

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

learner = PPO(
    policy=MlpPolicy,
    env=learned_reward_venv,
    seed=0,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
    n_steps=64,
)
learner.learn(1000)  # Note: set to 100000 to train a proficient expert

Then we can evaluate it using the original reward.

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

reward, _ = evaluate_policy(learner.policy, venv, 10)
print(reward)

/home/norabelrose/miniforge3/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


8.6
